<h1 align="center"><font color="yellow">Pytorch: Neural Networks</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

In [1]:
%load_ext watermark 
%watermark -v -p numpy,pandas,matplotlib,requests,torch

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.13.2

numpy     : 1.24.3
pandas    : 2.0.1
matplotlib: 3.7.1
requests  : 2.31.0
torch     : 2.0.1



In [1]:
import torch

# <font color="red">Modelos Sequenciais</font>

A forma mais simples de definir uma `Rede Neural` em `Pytorch` é utilizando a classe `Sequential`. Esta classe nos permite definir uma sequência de camadas, que se aplicaram de maneira sequencial (as saídas de uma camada serão a entrada da seguinte). 

In [2]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(torch.nn.Linear(D_in, H), # H --> Hidden
                            torch.nn.ReLU(),
                            torch.nn.Linear(H, D_out),
                           )


Este modelo anterior é um `MLP` com $784$ entradas, $100$ neurônios na camada oculta e $10$ saídas. Vejamos um exemplo de como calcular as saídas do Modelo a partir de umas entradas de exemplo:

In [3]:
outputs = model(torch.randn(64, 784))
outputs.shape


torch.Size([64, 10])

<font color="orange">É importante observar que os modelos de `Pytorch` (pelo geral) sempre esperam que a primeira dimensão seja a `Dimensão Batch`. Lembramos que treinar na `GPU` é assim:</font>

In [4]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Pegamos como exemplo: MNIST:

In [5]:
from sklearn.datasets import fetch_openml

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

/home/eddygiusepe/miniconda3/envs/LightningAI/lib/python3.9/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


((70000, 784), (70000,))

In [6]:
import numpy as np

# Normalização e Split:

X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(int), Y[60000:].astype(int)

In [7]:
# Função Loss e Derivada:

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)


def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [11]:
# Convertemos os dados a tensores e copiamos para a  gpu
X_t = torch.from_numpy(X_train.values).float().cuda()
Y_t = torch.from_numpy(y_train.values).long().cuda()

# Loop de Treinamento
epochs = 1000
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())
    
    # Zeramos os gradientes:
    model.zero_grad()

    # Backprop (calculamos todos os gradientes automáticamente)
    loss.backward()

    # update dos pesos:
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")


Epoch 10/1000 Loss 0.24230
Epoch 20/1000 Loss 0.23686
Epoch 30/1000 Loss 0.23204
Epoch 40/1000 Loss 0.22762
Epoch 50/1000 Loss 0.22352
Epoch 60/1000 Loss 0.21967
Epoch 70/1000 Loss 0.21605
Epoch 80/1000 Loss 0.21263
Epoch 90/1000 Loss 0.20938
Epoch 100/1000 Loss 0.20629
Epoch 110/1000 Loss 0.20334
Epoch 120/1000 Loss 0.20052
Epoch 130/1000 Loss 0.19781
Epoch 140/1000 Loss 0.19522
Epoch 150/1000 Loss 0.19273
Epoch 160/1000 Loss 0.19033
Epoch 170/1000 Loss 0.18801
Epoch 180/1000 Loss 0.18578
Epoch 190/1000 Loss 0.18363
Epoch 200/1000 Loss 0.18154
Epoch 210/1000 Loss 0.17953
Epoch 220/1000 Loss 0.17757
Epoch 230/1000 Loss 0.17568
Epoch 240/1000 Loss 0.17383
Epoch 250/1000 Loss 0.17204
Epoch 260/1000 Loss 0.17031
Epoch 270/1000 Loss 0.16861
Epoch 280/1000 Loss 0.16697
Epoch 290/1000 Loss 0.16536
Epoch 300/1000 Loss 0.16380
Epoch 310/1000 Loss 0.16227
Epoch 320/1000 Loss 0.16078
Epoch 330/1000 Loss 0.15933
Epoch 340/1000 Loss 0.15791
Epoch 350/1000 Loss 0.15652
Epoch 360/1000 Loss 0.15516
E

In [13]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test.values).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())


0.9745

# <font color="red">Otimizadores e Funções de Perda</font>

Vamos usar as funções que Pytorch nos facilita. Ver a [Documentação Pytorch](https://pytorch.org/docs/stable/index.html) para mais detalhes.

In [14]:
criterion = torch.nn.CrossEntropyLoss()


In [15]:
# Otimizadores em `torch.optim`

optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Nosso Loop de Treinamento fica mais compactado, assim:

In [18]:
model = torch.nn.Sequential(torch.nn.Linear(D_in, H),
                            torch.nn.ReLU(),
                            torch.nn.Linear(H, D_out),
                           ).to("cuda")


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # Zeramos os gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos os gradientes automáticamente)
    loss.backward()

    # update dos pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test.values).float().cuda())

print("")
print("\033[93mA accuracy é: \033[0m")
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.78106
Epoch 20/100 Loss 1.36611
Epoch 30/100 Loss 1.12115
Epoch 40/100 Loss 0.97920
Epoch 50/100 Loss 0.86094
Epoch 60/100 Loss 0.77130
Epoch 70/100 Loss 0.70736
Epoch 80/100 Loss 0.65396
Epoch 90/100 Loss 0.61035
Epoch 100/100 Loss 0.57421

A accuracy é: 


0.9341

# <font color="pink">Modelos customizados</font>

<font color="orange">Em muitos casos definir uma `Rede Neural` como uma sequência de camadas é suficiente, em outros casos será um fator limitante. `Um exemplo` são as Redes Residuais, nas que não só utilizamos a saída de uma camada para alimentar a seguinte senão que, ademais, sumamos sua própria entrada. Esse tipo de arquitetura não pode ser definida com a classe `Sequential`, para isso precisamos CUSTOMIZAR. Para isso `Pytorch` nos oferece a seguinte sintaxe:</font>

In [19]:
# Criamos uma classe que herda de `torch.nn.Module`

class Model(torch.nn.Module):
    
    # Construtor
    def __init__(self, D_in, H, D_out):
        
        # Chamamos ao construtor da classe pai
        super(Model, self).__init__()
        
        # Definimos nossa camadas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)
        
    # Lógica para calcular as saídas da Rede:
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    

In [20]:
model = Model(784, 100, 10)

outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

<font color="orange">Agora, podemos treinar a nossa Rede Neural da mesma forma que fizemos anteriormente:</font>

In [25]:
model.to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

epochs = 500
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # Função Forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test.values).float().cuda())
print("")

print("\033[93mA accuracy é: \033[0m")
accuracy_score(y_test, y_pred.cpu().numpy())


Epoch 10/500 Loss 0.24836
Epoch 20/500 Loss 0.24610
Epoch 30/500 Loss 0.38185
Epoch 40/500 Loss 0.35002
Epoch 50/500 Loss 0.32750
Epoch 60/500 Loss 0.31121
Epoch 70/500 Loss 0.29875
Epoch 80/500 Loss 0.28878
Epoch 90/500 Loss 0.28052
Epoch 100/500 Loss 0.27350
Epoch 110/500 Loss 0.26739
Epoch 120/500 Loss 0.26199
Epoch 130/500 Loss 0.25715
Epoch 140/500 Loss 0.25275
Epoch 150/500 Loss 0.24872
Epoch 160/500 Loss 0.24499
Epoch 170/500 Loss 0.24153
Epoch 180/500 Loss 0.23828
Epoch 190/500 Loss 0.23522
Epoch 200/500 Loss 0.23232
Epoch 210/500 Loss 0.22958
Epoch 220/500 Loss 0.22696
Epoch 230/500 Loss 0.22446
Epoch 240/500 Loss 0.22206
Epoch 250/500 Loss 0.21976
Epoch 260/500 Loss 0.21754
Epoch 270/500 Loss 0.21541
Epoch 280/500 Loss 0.21335
Epoch 290/500 Loss 0.21135
Epoch 300/500 Loss 0.20942
Epoch 310/500 Loss 0.20755
Epoch 320/500 Loss 0.20573
Epoch 330/500 Loss 0.20397
Epoch 340/500 Loss 0.20225
Epoch 350/500 Loss 0.20058
Epoch 360/500 Loss 0.19896
Epoch 370/500 Loss 0.19737
Epoch 380/

0.963

<font color="orange">A seguir vamos ver outro exemplo de como definir `MLP` com conexões residuais, algo que podemos fazer simplesmente usando um Modelo sequential:</font>